In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem.lancaster import LancasterStemmer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("Crowler/booking_reviews3.csv")

In [3]:
data[:5]

,title,pos_text,neg_text,ratingValue,bestRating
0,наступної мандрівки до Львова оберемо цей готель.,Відмінне розташування готелю! Дуже близько до ...,"один маленький мінус, але нам це не завдало не...",10.0,10.0
1,"Сніданок - великі порціїї, смачно!","Сніданок - великі порціїї, смачно! 1 страва бе...",Сам готель усередині нагадує гуртожиток чи хос...,6.3,10.0
2,"Все чудово! За нагоди, обов'язково знову завіт...",Дуже чудовий і затишний отель! Дуже зручне роз...,"Сантехніку (труби , стоки) необхідно перевірит...",9.6,10.0
3,"класне розташування, такий собі острівець тиші...","класне розташування, такий собі острівець тиші...",начебто новий ремонт - а велика тріщина над дв...,7.5,10.0
4,"Гарне місце для туристів або відрядження, але ...","Ідеальний номер, як для тризіркового готелю - ...",Дуже круті сходи. Складно знайти поруч місце д...,9.2,10.0


In [4]:
def read_stop_words(file):
    with open(file) as f:
        stop_words = f.read().split('\n')

    return stop_words

In [5]:
stop_words = read_stop_words('ukrainian-stopwords.txt')

In [6]:
ignore_symbols = [',', '.', '!', '?']

In [7]:
ignore = ignore_symbols + stop_words

In [8]:
stemmer = LancasterStemmer()

In [9]:
def add_words_from_sentence_to_bag_of_words(sentence : str, ignore_words: list, bag_of_words : set):
    words = nltk.word_tokenize(sentence)
    for word in words:
        if word in ignore_words:
            continue
        bag_of_words.add(stemmer.stem(word.lower()))

In [10]:
bag_of_words = set()
for i in range(0, len(data)):
    add_words_from_sentence_to_bag_of_words(data['title'][i], ignore, bag_of_words)
    add_words_from_sentence_to_bag_of_words(data['pos_text'][i], ignore, bag_of_words)
    add_words_from_sentence_to_bag_of_words(data['neg_text'][i], ignore, bag_of_words)

In [11]:
def create_bag_of_words_vector(sentence: str, ignore_words : list, bag_of_words : set):
    d = dict.fromkeys(bag_of_words, 0) 
    words = nltk.word_tokenize(sentence)
    for word in words:
        if word in d:
            d[word] += 1
            
    return list(d.values())

In [12]:
X, y = [], []
for i in range(0, len(data)):
    title = create_bag_of_words_vector(data['title'][i], ignore, bag_of_words)
    pos_text = create_bag_of_words_vector(data['pos_text'][i], ignore, bag_of_words)
    neg_text = create_bag_of_words_vector(data['neg_text'][i], ignore, bag_of_words)
    rating_value = data['ratingValue'][i]
    xi = title + pos_text + neg_text
    xi = np.array(title) + np.array(pos_text) + np.array(neg_text)
    yi = int(round(rating_value / 2))
    
    X.append(xi)
    y.append(yi)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [14]:
clf = LogisticRegression(random_state=42, solver='lbfgs', multi_class='multinomial')

In [15]:
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=42, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

### Metric 

In [16]:
from sklearn.metrics import classification_report

In [17]:
y_pred = clf.predict(X_test) 

In [18]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         4
           2       0.67      0.14      0.24        14
           3       0.30      0.20      0.24        30
           4       0.45      0.42      0.43       110
           5       0.64      0.75      0.69       183

   micro avg       0.56      0.56      0.56       341
   macro avg       0.41      0.30      0.32       341
weighted avg       0.54      0.56      0.54       341



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
